# BLAST method for database searching
This practical uses the simple BLAST algorithm 'MyBlast' from Chapter 7 of Rocha and Ferreira (2018) Bioinformatics Algorithms. 

It is not supposed to be a fully-fledged program but aims only to show the initial steps in the database search method. 

The first step is to create a database. This will be a text file containing a set of strings representing the sequencees to be searched. Each sequence is short enough in the test data to be given as a single line in the file. 

The authors provide a test data file callled seqBlast.txt

The function read_database opens that file and creates the database.

In [1]:
def read_database (filename):    
    """"
    reads the sequences to search from a text file 
    """
    file = open (filename)
    db = []
    for line in file:
        db.append(line.rstrip())
    file.close()
    return db

The next step is to pre-process the query sequence. The authors call this producing a 'map' of all word of a particular size (an adjustable parameter here - although remember that larger words are more demanding). A table of these can be a Python dictionary - as that naturally reproduces the hashing method described in the lecture. 

In [2]:
def build_map (query, w):
    """
    preprocesses the query to store the positions of words
    """
    res = {}
    for i in range(len(query)-w+1):
        subseq = query[i:i+w]
        if subseq in res:
            res[subseq].append(i)
        else:
            res[subseq] = [i]
    return res

In the authors' simplified version of the algorithm they do not use a substitution table but instead a simple scoring function similar to the one used for the Smith-Waterman example. The function gives a positive score only to perfect hits. As explained in the lecture this has to be the default for many protein sequence words in BLOSUM62 as well. 

The next function get_hits will scan a sequence and find all matches of the words from the query map. These are considered 'hits'.

In [3]:
def get_hits (seq, map, w):
    """
    scans the sequence for word hit in map
    returns tuples: 
    index of match in query with the
    index of match in sequence
    """    
    res = []  # list of tuples
    for i in range(len(seq)-w+1):
        subseq = seq[i:i+w]
        if subseq in map:
            l = map[subseq]
            for ind in l:
                res.append( (ind,i) )
    return res

The next step is to extend the hits found by the previous function. 

Here the the hit is extended while the new aligned positions score greated than or equal to half of the new positions.

No gaps are used. But a check needs to be made that the extension has not reached the end of either the query or the sequence. 

In [4]:
def extends_hit (seq, hit, query, w):
    """
    the hit positions are extended
    based on the sequence and query matches
    on either side
    """    
    stq, sts = hit[0], hit[1]
    ## extend hit forward
    matfw = 0       
    k=0
    bestk = 0
    while 2*matfw >= k and stq+w+k < len(query) and sts+w+k < len(seq):
        if query[stq+w+k] == seq[sts+w+k]: 
            matfw+=1
            bestk = k+1
        k += 1
    size = w + bestk
    ## extend hit backwards
    k = 0
    matbw = 0   
    bestk = 0
    while 2*matbw >= k and stq > k and sts > k:
        if query[stq-k-1] == seq[sts-k-1]: 
            matbw+=1
            bestk = k+1
        k+=1       
    size += bestk
    
    return (stq-bestk, sts-bestk, size, w+matfw+matbw)


The output for the extended hit is an alignment starting positions in the query and in the sequence, its size and finally a count of identity matches added to the word length. The words from hits are all fully identical by definition in this implementation. The identity count is a measure of the final score for the extended alignment.

The extend_hit function is applied to all the hits in turn by the following function hit_best_score. It then returns the top scoring one. 

In [5]:
def hit_best_score(seq, query, m, w):
    """
    the hit positions are extended
    based on the sequence and query matches
    on either side
    """   
    hits = get_hits(seq, m, w)
    bestScore = -1.0
    best = ()
    for h in hits:
        ext = extends_hit(seq, h, query, w)
        score = ext[3]
        if score > bestScore or (score== bestScore and ext[2] < best[2]):
            bestScore = score
            best = ext
    return best

The final step is to apply the previous functions to compare the query with all the sequences in the database. The best overall alignment is found for sequences with hits. The result is a tuple similar to the ones above. 

In [6]:
def best_alignment (db, query, w):
    """
    compare the query with all the sequences
    in the database 
    all significant scores >=0 are returned
    """    
    m = build_map(query, w)
    bestScore = -1.0
    res = (0,0,0,0,0)
    for k in range(0,len(db)):
        bestSeq = hit_best_score(db[k], query, m, w)
        if bestSeq != ():
            score = bestSeq[3]  
            if score > bestScore or (score== bestScore and bestSeq[2] < res[2]):
                bestScore = score
                res = bestSeq[0], bestSeq[1], bestSeq[2], bestSeq[3], k
    if bestScore < 0: return ()
    else: return res

Applying it to a database from the test data file.  The standard DNA word length of 11 is used.

In [10]:
db = read_database("C:\\Users\\mfs1001\\Desktop\\seqBlast.txt")
query = "gacgcctcgcgctcgcgcgctgaggcaaaaaaaaaaaaaaaaaaaatcggatagctagctgagcgctcgatagcgcgttcgctgcatcgcgtatagcgctgaagctcccggcgagctgtctgtaaatcggatctcatctcgctctatcct"
r = best_alignment(db, query, 11)
print(r)

(1, 38, 149, 108, 3)


In [11]:
query2 = "cgacgacgacgacgaatgatg"
r = best_alignment(db, query2, 11)
print(r)

(0, 0, 21, 21, 4)
